In [1]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from keras.models import Model
from keras.utils import plot_model

Using TensorFlow backend.


## Load data

In [2]:
sequence = np.array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])
n_in = len(sequence)
sequence = sequence.reshape((1, n_in, 1))

## Model

In [7]:
model = Sequential()
model.add(LSTM(100, activation='relu', input_shape=(n_in,1)))
model.add(RepeatVector(n_in))
model.add(LSTM(100, activation='relu', return_sequences=True))
model.add(TimeDistributed(Dense(1)))
model.compile(optimizer='adam', loss='mse')

print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 100)               40800     
_________________________________________________________________
repeat_vector_2 (RepeatVecto (None, 9, 100)            0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 9, 100)            80400     
_________________________________________________________________
time_distributed_2 (TimeDist (None, 9, 1)              101       
Total params: 121,301
Trainable params: 121,301
Non-trainable params: 0
_________________________________________________________________
None


In [10]:
model.fit(sequence, sequence, epochs=300, verbose=0)

In [5]:
yhat = model.predict(sequence, verbose=0)
print(yhat[0,:,0])

[0.1043822  0.19986954 0.29883394 0.39925766 0.49997026 0.6004914
 0.7004725  0.7996464  0.8999917 ]


## Keep Standalone LSTM Encoder

In [6]:
# connect the encoder LSTM as the output layer
encoder_model = Model(inputs=model.inputs, outputs=model.layers[0].output)
plot_model(encoder_model, show_shapes=True, to_file='lstm_encoder.png')
# get the feature vector for the input sequence
yhat_enc = encoder_model.predict(sequence)
print(yhat_enc.shape)
print(yhat_enc)

(1, 100)
[[0.0776437  0.0783195  0.         0.07300517 0.         0.
  0.         0.         0.         0.08395053 0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.04406476 0.04826838 0.
  0.         0.         0.         0.10367119 0.08991672 0.
  0.061174   0.         0.         0.         0.06285162 0.06706778
  0.04621964 0.         0.         0.07090113 0.01700027 0.
  0.         0.         0.05734985 0.         0.         0.
  0.         0.10045734 0.06470631 0.         0.07695849 0.00309676
  0.         0.         0.08872069 0.         0.         0.
  0.         0.10259909 0.         0.         0.         0.
  0.03834438 0.         0.         0.         0.07546712 0.
  0.07219782 0.09610678 0.         0.         0.         0.13336232
  0.         0.1337662  0.         0.08724104 0.05531295 0.06848243
  0.         0.         0.04978506 0.03069826 0.         0.09301534
  0.         0.06641914 0.         0.         0.   

## References

1. [Machine Learning Mastery: A Gentle Introduction to LSTM Autoencoders](https://machinelearningmastery.com/lstm-autoencoders/)